In [ ]:
# requruiments

!pip install requests
!pip install time
!pip install pandas
!pip install numpy
!pip install matplotlib.pyplot
!pip install schedule

In [ ]:
import requests
import time

def fetch_data():
    coin_id = "bitcoin"
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range"

    to_ts = int(time.time())                 
    from_ts = to_ts - 90 * 24 * 60 * 60    

    params = {    
        "vs_currency": "usd",
        "from": from_ts,
        "to": to_ts        
    }

    response = requests.get(url, params=params)
    data = response.json()

    # print(data)

    return data

In [ ]:
import pandas as pd
import numpy as np

data = fetch_data()

def normalize_coingecko_series(data, field="prices", col="BTC"):
    df = pd.DataFrame(data[field], columns=["ts_ms", col])
    df["date"] = pd.to_datetime(df["ts_ms"], unit="ms", utc=True)
    df = df.drop(columns=["ts_ms"]).drop_duplicates(subset=["date"]).sort_values("date")
    df[col] = pd.to_numeric(df[col], errors="coerce")
    df[col] = df[col].replace(0, np.nan)
    df[col] = df[col].ffill(limit=3)
    return df[["date", col]]

btc_df = normalize_coingecko_series(data, col="BTC")
btc_df.head()

def to_daily_close(df, value_col):
    s = df.set_index("date")[value_col].sort_index()
    daily = s.resample("1D").last().ffill(limit=2)
    return daily.reset_index().rename(columns={value_col: value_col})

btc_daily = to_daily_close(btc_df, "BTC")
btc_daily.head()

In [ ]:
import matplotlib.pyplot as plt

def plot_graph(btc_daily):
    plt.figure(figsize=(10,5))
    plt.plot(btc_daily["date"], btc_daily["BTC"], color="orange", label="BTC Daily Close")
    plt.title("Bitcoin Daily Closing Prices")
    plt.xlabel("Date")
    plt.ylabel("Price (USD)")
    plt.legend()
    plt.show()

In [ ]:
import schedule
import requests
import time
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def schedule_graph():
    data = fetch_data()
    btc_df = normalize_coingecko_series(data, col="BTC")
    btc_daily = to_daily_close(btc_df, "BTC")
    plot_graph(btc_daily)
    
    

schedule.every().day.at("22:39").do(schedule_graph)
print("Scheduler started. Will fetch and graph data every day at 09:00.")

while True:
    schedule.run_pending()
    time.sleep(60)